In [9]:
from IPython.display import Image

- N 张卡组成一个 ring 环，计算步数，2(N-1)
    - scatter-reduce: (N-1)，非标准 nccl
    - all-gather: (N-1)
- 3张卡，长度为6的向量加和为例；
    - input (each gpu model gradients):
        - `[a0, a1 | a2, a3 | a4, a5] = [A0 | A1 | A2]`
        - `[b0, b1 | b2, b3 | b4, b5] = [B0 | B1 | B2]`
        - `[c0, c1 | c2, c3 | c4, c5] = [C0 | C1 | C2]`
    - output (sync model gradients across gpus):
        - `[a0+b0+c0, a1+b1+c1, a2+b2+c2, a3+b3+c3, a4+b4+c4, a5+b5+c5]`
        - `[A0 + B0 + C0 | A1 + B1 + C1 | A2 + B2 + C2]`

### torch scatter reduce

- https://pytorch.org/docs/stable/generated/torch.Tensor.scatter_reduce_.html

In [1]:
import torch

In [2]:
src = torch.tensor([1., 2., 3., 4., 5., 6.])
index = torch.tensor([0, 1, 0, 1, 2, 1])
input = torch.tensor([1., 2., 3., 4.])
input.scatter_reduce(0, index, src, reduce="sum", include_self=True)

tensor([ 5., 14.,  8.,  4.])

In [3]:
1+(1+3), 2+(2+4+6), 3+(5), 4

(5, 14, 8, 4)

In [4]:
src = torch.tensor([1., 2., 3., 4., 5., 6.])
index = torch.tensor([0, 1, 0, 1, 2, 1])
input = torch.tensor([1., 2., 3., 4.])
input.scatter_reduce(0, index, src, reduce="mean", include_self=True)

tensor([1.6667, 3.5000, 4.0000, 4.0000])

In [5]:
(1+(1+3))/3, (2+(2+4+6))/4, (3+(5))/2, 4

(1.6666666666666667, 3.5, 4.0, 4)

### phase1： scatter reduce

> 减少通信量；

- `[a0, a1 | a2, a3 | a4, a5] = [A0 | A1 | A2]`
- `[b0, b1 | b2, b3 | b4, b5] = [B0 | B1 | B2]`
- `[c0, c1 | c2, c3 | c4, c5] = [C0 | C1 | C2]`
- scatter：data chunks，reduce：规约（降维）
    - nccl 是 reduce-scatter
- step1
    - GPU0 =>(A2) GPU1 =>(B0) GPU2 =>(C1) GPU0
        - GPU0: A1 + C1, `[A0, A1+C1, A2]`
        - GPU1: B2 + A2, `[B0, B1, B2+A2]`
        - GPU2: C0 + B0, `[C0+B0, C1, C2]` 
- step2
    - GPU0 =>(A1+C1) GPU1 =>(B2+A2) GPU2 =>(C0+B0) GPU0
        - GPU0: `[C0+B0+A0, A1+C1, A2]`
        - GPU1: `[B0, A1+C1+B1, B2+A2]`
        - GPU2: `[C0+B0, C1, B2+A2+C2]`

### phase2: all-gather

- `S0: A0+B0+C0, S1: A1+B1+C1, S2: A2+B2+C2`
- step1:
    - GPU0 =>(S0) GPU1 =>(S1) GPU2 =>(S2) GPU0
      - GPU0: [S0, ..., S2]
      - GPU1: [S0, S1, ...]
      - GPU2: [..., S1, S2]
- step2:
    - GPU0 =>(S2) GPU1 =>(S0) GPU2 =>(S1) GPU0
        - GPU0: [S0, S1, S2]
        - GPU1: [S0, S1, S2]
        - GPU2: [S0, S1, S2]

In [10]:
Image(url='./imgs/ring-allreduce.png', width=600)

### why ring-allreduce

- 高效的带宽利用率 (Efficient Bandwidth Utilization):
    - 分块传输: Ring-AllReduce 将需要同步的数据（例如梯度）分成多个小块（chunks）。
    - 流水线效应: 数据块在环上逐步传输和计算。一个 GPU 可以同时发送一个块给下一个节点，并从上一个节点接收另一个块。这种流水线方式使得 GPU 间的通信链路（如 NVLink 或网络带宽）能够持续被利用，而不是在等待整个大块数据传输完成。
    - 点对点通信: 每个 GPU 只需与其在环中的直接邻居通信。这使得算法可以充分利用现代 GPU 系统中高速的点对点连接（如 NVLink），避免了所有 GPU 都向一个中心点发送数据可能造成的拥塞。理论上，在 N 个 GPU 的环中，每个 GPU 在 Scatter-Reduce 和 All-Gather 阶段总共发送和接收的数据量大约是 2 * (N-1)/N * TotalDataSize，接近于最优值 2 * TotalDataSize。
- 均衡的通信负载 (Balanced Communication Load):
    - 在 Ring-AllReduce 中，每个 GPU 发送和接收的数据量大致相同，计算负载（Reduce 操作）也相对均衡地分布在各个步骤中。
    - 这避免了像基于树（Tree-based）的 All-Reduce 算法中可能出现的根节点通信瓶颈问题，因为在树形结构中，靠近根节点的 GPU 需要处理更多的数据聚合或分发任务。
- 避免中心瓶颈 (Avoids Central Bottleneck):
    - 与参数服务器（Parameter Server）架构或其他需要中心协调节点的同步方法不同，Ring-AllReduce 是完全去中心化的。没有单个节点会成为性能瓶颈或单点故障。
- 良好的可扩展性 (Good Scalability):
    - 虽然完成一次完整的 Ring-AllReduce 需要 2 * (N-1) 步（N 是 GPU 数量），延迟会随着 N 线性增加，但关键在于每个 GPU 的带宽需求基本保持不变（与 N 无关）。
对于带宽是主要瓶颈的大规模系统（尤其是在传输大量梯度时），这种恒定的带宽需求使得 Ring-AllReduce 比那些带宽需求随节点数增加而增加的算法更具扩展性。